## Step 1 - Load Dataset

Pada tahap ini dataset dimuat untuk memastikan data siap digunakan dalam proses preprocessing dan pemodelan. Tujuannya adalah memverifikasi bahwa data terbaca dengan benar sebelum analisis lanjutan dilakukan.

In [7]:
import pandas as pd

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 50)

df = pd.read_csv("../data/Heart_Disease_Prediction.csv")
df.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


## Step 2 - Memisahkan Fitur dan Target

Pada tahap ini dataset dibagi menjadi dua bagian utama:

- **Fitur (X)** → seluruh kolom yang digunakan sebagai input model
- **Target (y)** → kolom label yang akan diprediksi oleh model

Kolom **"Heart Disease"** dipisahkan dari dataset fitur agar model dapat belajar memprediksi kondisi penyakit jantung berdasarkan variabel lainnya.

In [16]:
X = df.drop(columns=["Heart Disease"])
y = df["Heart Disease"]

X.head(), y.to_frame().head()

(   Age  Sex  Chest pain type   BP  Cholesterol  FBS over 120  EKG results  Max HR  Exercise angina  ST depression  Slope of ST  Number of vessels fluro  Thallium
 0   70    1                4  130          322             0            2     109                0            2.4            2                        3         3
 1   67    0                3  115          564             0            2     160                0            1.6            2                        0         7
 2   57    1                2  124          261             0            0     141                0            0.3            1                        0         7
 3   64    1                4  128          263             0            0     105                1            0.2            2                        1         7
 4   74    0                2  120          269             0            2     121                1            0.2            1                        1         3,
   Heart Disease
 0  

## Step 3 - Encoding Target

Model machine learning hanya dapat memproses nilai numerik. Oleh karena itu, label pada kolom **"Heart Disease"** diubah menjadi:

- **Presence** → 1
- **Absence** → 0

Proses ini disebut **encoding target**, agar model dapat mengenali dan mempelajari pola dari label tersebut.

In [9]:
y = y.map({"Absence": 0, "Presence": 1})
y.value_counts()

Heart Disease
0    150
1    120
Name: count, dtype: int64

## Step 4 - Preprocessing Fitur

Pada tahap ini fitur dipersiapkan agar dapat diproses oleh model machine learning.

- Fitur numerik dinormalisasi menggunakan **StandardScaler** agar seluruh variabel berada pada skala yang sebanding.
- Fitur kategorikal diubah menjadi format numerik menggunakan **One-Hot Encoding**.

Tujuan preprocessing ini adalah agar model dapat memahami seluruh fitur dengan konsisten dan menghasilkan performa prediksi yang optimal.

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [18]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

numeric_features, categorical_features

(['Age',
  'Sex',
  'Chest pain type',
  'BP',
  'Cholesterol',
  'FBS over 120',
  'EKG results',
  'Max HR',
  'Exercise angina',
  'ST depression',
  'Slope of ST',
  'Number of vessels fluro',
  'Thallium'],
 [])

In [19]:
categorical_features = [
    "Sex",
    "Chest pain type",
    "FBS over 120",
    "EKG results",
    "Exercise angina",
    "Slope of ST",
    "Thallium"
]

numeric_features = [
    col for col in X.columns if col not in categorical_features
]

numeric_features, categorical_features

(['Age',
  'BP',
  'Cholesterol',
  'Max HR',
  'ST depression',
  'Number of vessels fluro'],
 ['Sex',
  'Chest pain type',
  'FBS over 120',
  'EKG results',
  'Exercise angina',
  'Slope of ST',
  'Thallium'])

### Pemisahan Fitur Numerik dan Kategorikal

Pada tahap ini, fitur dipisahkan menjadi dua kelompok utama:

- **Fitur Kategorikal** ditentukan secara manual berdasarkan karakterisktik data.
- **Fitur Numerik** diperoleh secara otomatis dari sisa kolom yang tidak termasuk fitur kategorikal.

Pendekatan ini memastikan setiap jenis fitur dapat diproses menggunakan metode preprocessing yang sesuai sebelum digunakan pada model machine learning.

## Step 5 - Preprocessing Pipeline

Pada tahap ini, fitur numerik dinormalisasi menggunakan StandardScaler, dan fitur kategorikal diubah menjadi format numerik menggunakan OneHotEncoder. Tujuannya adalah agar seluruh fitur memiliki skala yang konsisten sebelum masuk ke proses pelatihan model.

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Membuat pipeline preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['Age', 'BP', 'Cholesterol', 'Max HR',
                                  'ST depression', 'Number of vessels fluro']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Sex', 'Chest pain type', 'FBS over 120',
                                  'EKG results', 'Exercise angina',
                                  'Slope of ST', 'Thallium'])])

## Step 6 - Membagi Data Latih dan Data Uji (Train-Test Split)

Di tahap ini, dataset dipisahkan menjadi dua bagian:

- **Data latih (X_train, y_train)**: dipakai untuk "ngajar" model.
- **Data uji (X_test, y_test)**: dipakai buat ngetes model pada data yang belum pernah dia lihat.

Kenapa harus dipisah?
Supaya kita bisa menilai model secara adil. Kalau semua data dipakai buat latihan, model bisa terlihat jago padahal cuma **ngafalin** data (overfitting).

Konfigurasi yang dipakai:
- **80% data latih**
- **20% data uji**
- 'random_state=42' supaya hasil split konsisten (kalau diulang, hasilnya sama).
- 'stratify=y' supaya proporsi kelas target tetap seimbang di train dan test.

In [23]:
from sklearn.model_selection import train_test_split

# Membagi dataset: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Cek ukuran hasil split biar yakin tidak ketuker
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)

# Optimal: cek proporsi kelas di train vs test (biar yakin stratify jalan)
print("\nProporsi kelas (train):")
print(y_train.value_counts(normalize=True))

print("\nProporsi kelas (test):")
print(y_test.value_counts(normalize=True))

X_train: (216, 13)
X_test : (54, 13)
y_train: (216,)
y_test : (54,)

Proporsi kelas (train):
Heart Disease
Absence     0.555556
Presence    0.444444
Name: proportion, dtype: float64

Proporsi kelas (test):
Heart Disease
Absence     0.555556
Presence    0.444444
Name: proportion, dtype: float64
